# Assignment 7- Snehal Thorat

### Question 1 (30 points)
1. Create Spark session as we did in the class. Make sure you enable Hive support.
2. Load all files from "data\retail-data\by-day" into a single Spark DataFrame. There should be 300+ files in
the folder.
3. Print Schema of the DF created in 1.1
4. Create SQL temp table to be used in Spark SQL for the current session. Name table "RetailData"
5. Display list of Hive Tables, permanent and temporary in the "default" schema


In [139]:
from pyspark.sql.functions import instr,col
try:
    spark.stop()
except:
    pass
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("assign7").enableHiveSupport().getOrCreate()

In [140]:
spark

In [141]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/mnt/hgfs/Shared_folder/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*")


In [142]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [143]:
df.createOrReplaceTempView("RetailData")

In [144]:
spark.sql("show tables").show()

+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
| default|   business|      false|
| default|business_pq|      false|
| default|  people_pq|      false|
| default|     review|      false|
| default|  review_pq|      false|
| default|  sample_07|      false|
| default|  sample_08|      false|
| default|     u_data|      false|
| default|  u_data_pq|      false|
| default|     u_item|      false|
| default|  u_item_pq|      false|
| default|     u_user|      false|
| default|  u_user_pq|      false|
| default|      users|      false|
| default|   users_pq|      false|
|        | retaildata|       true|
+--------+-----------+-----------+



### Question 2 (30 points)


1) Cache the DF so that it will reside in memory after first "action".


In [145]:
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

2) Perform count() function on the DataFrame to cache the DF


In [146]:
df.count()

541909

3) Use limit function to display first five records using both SQL and DF API. Full content of each column
should be displayed, use proper parameter to show() function


In [147]:
spark.sql("SELECT * FROM retaildata limit 5").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+



In [148]:
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



### Question 3 (60 points)


Use Spark API and Spar SQL (two different commands) to find how many records are present in the DF loaded
in "1" for every year-month combination. Sort by year-month in default sort order.
1. Hint 1 - use concat, year, month functions for DF API
2. Hint 2 - use date_format for Spark SQL


In [149]:
spark.sql("SELECT YEAR(InvoiceDate), MONTH(InvoiceDate),COUNT(*) FROM retaildata GROUP BY YEAR(InvoiceDate), MONTH(InvoiceDate) ORDER BY YEAR(InvoiceDate), MONTH(InvoiceDate)").show()

+-------------------------------+--------------------------------+--------+
|year(CAST(InvoiceDate AS DATE))|month(CAST(InvoiceDate AS DATE))|count(1)|
+-------------------------------+--------------------------------+--------+
|                           2010|                              12|   42481|
|                           2011|                               1|   35147|
|                           2011|                               2|   27707|
|                           2011|                               3|   36748|
|                           2011|                               4|   29916|
|                           2011|                               5|   37030|
|                           2011|                               6|   36874|
|                           2011|                               7|   39518|
|                           2011|                               8|   35284|
|                           2011|                               9|   50226|
|           

In [150]:
from pyspark.sql import functions as F
df_year_month = df.withColumn('Year', F.split(df.InvoiceDate, '-')[0]).withColumn('Month', F.split(df.InvoiceDate, '-')[1])
df_year_month.groupBy("Year","Month").count().orderBy("Year","Month").show()

+----+-----+-----+
|Year|Month|count|
+----+-----+-----+
|2010|   12|42481|
|2011|   01|35147|
|2011|   02|27707|
|2011|   03|36748|
|2011|   04|29916|
|2011|   05|37030|
|2011|   06|36874|
|2011|   07|39518|
|2011|   08|35284|
|2011|   09|50226|
|2011|   10|60742|
|2011|   11|84711|
|2011|   12|25525|
+----+-----+-----+



### Question 4 (80 points) - Spark API (not SQL)


1) How many records have Null in field "Description"?


In [151]:
df.filter(col("description").isNull()).count()

1454

2) Create new DF and populate field "Description" with 'NULL-STR string.


In [152]:
#First let's clean up the DF: Replace two Spaces with one Space
from pyspark.sql.functions import regexp_replace, col
df = df.withColumn('Clean_Desc',regexp_replace(col("Description"), "  ", " "))

In [153]:
from pyspark.sql.functions import coalesce, lit
df1 = df.withColumn("Clean_Desc",coalesce(col("Description"),lit("NULL-STR")))
df1 = df1.drop('Description')
df1.show(1)
         

+---------+---------+--------+-------------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|        Clean_Desc|
+---------+---------+--------+-------------------+---------+----------+--------------+------------------+
|   580538|    23084|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|RABBIT NIGHT LIGHT|
+---------+---------+--------+-------------------+---------+----------+--------------+------------------+
only showing top 1 row



3) Create new Boolean column for each item in the following list:
['lunch','dinner','supper','breakfast']


In [154]:
#is_lunch|is_dinner|is_supper|is_breakfast

df4 = df1.withColumn('is_Lunch',df1.Clean_Desc.contains('LUNCH')).withColumn('is_dinner',df1.Clean_Desc.contains('DINNER')).withColumn('is_supper',df1.Clean_Desc.contains('SUPPER')).withColumn('is_breakfast',df1.Clean_Desc.contains('BREAKFAST'))

df4.head()

Row(InvoiceNo='580538', StockCode='23084', Quantity=48, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.79, CustomerID=14075.0, Country='United Kingdom', Clean_Desc='RABBIT NIGHT LIGHT', is_Lunch=False, is_dinner=False, is_supper=False, is_breakfast=False)

In [155]:
df4.columns

['InvoiceNo',
 'StockCode',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'Clean_Desc',
 'is_Lunch',
 'is_dinner',
 'is_supper',
 'is_breakfast']

4) Populate the column as in the example below, and output first 50 records that had any of the items in
the item list matched to the content of the column "Description".


In [158]:
df4.where((col("is_Lunch") == "True") | (col("is_dinner") == "True")| (col("is_supper") == "True")| (col("is_breakfast") == "True")).show(50)

+---------+---------+--------+-------------------+---------+----------+--------------+--------------------+--------+---------+---------+------------+
|InvoiceNo|StockCode|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|          Clean_Desc|is_Lunch|is_dinner|is_supper|is_breakfast|
+---------+---------+--------+-------------------+---------+----------+--------------+--------------------+--------+---------+---------+------------+
|   580540|    22109|       4|2011-12-05 08:49:00|     3.75|   13417.0|United Kingdom|FULL ENGLISH BREA...|   false|    false|    false|        true|
|   580545|    23209|      10|2011-12-05 09:16:00|     1.65|   12782.0|      Portugal|LUNCH BAG VINTAGE...|    true|    false|    false|       false|
|   580545|    23583|      10|2011-12-05 09:16:00|     1.65|   12782.0|      Portugal|LUNCH BAG PAISLEY...|    true|    false|    false|       false|
|   580595|    22383|      10|2011-12-05 10:38:00|     1.65|   14194.0|United Kingdom|LUNCH BAG SUKI

#### 2) Output frequencies for each Boolean column you have added in 4.1
Note: make sure you take care of the null values in the description field. The output combinations should
be either true or false .


In [159]:
df4.groupBy("is_Lunch").count().show()

+--------+------+
|is_Lunch| count|
+--------+------+
|    true| 18525|
|   false|523384|
+--------+------+



In [160]:
df4.groupBy("is_dinner").count().show()

+---------+------+
|is_dinner| count|
+---------+------+
|     true|  1746|
|    false|540163|
+---------+------+



In [161]:
df4.groupBy("is_supper").count().show()

+---------+------+
|is_supper| count|
+---------+------+
|    false|541909|
+---------+------+



In [162]:
df4.groupBy("is_breakfast").count().show()

+------------+------+
|is_breakfast| count|
+------------+------+
|        true|  1171|
|       false|540738|
+------------+------+

